In [5]:
import torch
import os
import imageio
import numpy as np
import torch
from rlogist.sasha.wsi_core.WholeSlideImage import WholeSlideImage
from tqdm import tqdm

ModuleNotFoundError: No module named 'rlogist'

In [6]:
import h5py
import numpy as np

# Path to the .h5 file
h5_file_path = "/home/tarun/rlogist/ACMIL/CAMELYON16_features/VIT_medical_ssl_level_1_3/lr/h5_files/patch_feats_pretrain_medical_ssl.h5"

# Open the .h5 file
with h5py.File(h5_file_path, "r") as h5_file:
    min_patches = float("inf")
    min_wsi = None

    for wsi_name in h5_file.keys():
        if "tumor" in wsi_name.lower() and "coords" in h5_file[wsi_name]:  # Check if "tumor" is in name
            num_patches = h5_file[wsi_name]["coords"].shape[0]  # Number of coordinate entries
            if num_patches < min_patches:
                min_patches = num_patches
                min_wsi = wsi_name

    if min_wsi is not None:
        print(f"WSI with the minimum number of patches (containing 'tumor'): {min_wsi} ({min_patches} patches)")
    else:
        print("No WSI with 'tumor' in the name found.")

WSI with the minimum number of patches (containing 'tumor'): tumor_038 (140 patches)


In [7]:
import openslide
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from shapely.geometry import Polygon
from matplotlib.patches import Polygon as MplPolygon

# Paths
wsi_path = "/media/internal_8T/karm/karm_8T_backup/camelyon16/images/all/test_010.tif"
xml_path = "/media/internal_8T/karm/karm_8T_backup/camelyon16/annotations/test_010.xml"  # Assuming annotations are in XML

# Load WSI
slide = openslide.OpenSlide(wsi_path)

# Get Level 3 (5X) properties
level = 3
downscale_factor = slide.level_downsamples[level]  # Scaling factor from level 0 to level 3
wsi_size = slide.level_dimensions[level]  # (width, height) at level 3

# Read WSI at level 3
wsi_img = slide.read_region((0, 0), level, wsi_size).convert("RGB")

# Parse XML for annotation coordinates
tree = ET.parse(xml_path)
root = tree.getroot()

annotations = []
for annotation in root.findall(".//Annotation"):
    coords = []
    for coord in annotation.findall(".//Coordinate"):
        x = float(coord.get("X")) / downscale_factor  # Scale to level 3
        y = float(coord.get("Y")) / downscale_factor  # Scale to level 3
        coords.append((x, y))
    annotations.append(Polygon(coords))  # Store as Shapely Polygon

# Plot the WSI with annotations
fig, ax = plt.subplots(figsize=(15, 15))
ax.imshow(wsi_img)

# Add Annotations
for polygon in annotations:
    patch = MplPolygon(np.array(polygon.exterior.coords), edgecolor='r', fill=False, linewidth=2)
    ax.add_patch(patch)

ax.set_title(f"WSI with Annotations at Level {level} (5X)")
ax.axis("off")
plt.show()

ModuleNotFoundError: No module named 'openslide'

In [5]:
action_sequences = torch.load('/media/internal_8T/naman/rlogist/images/combined_patches_frac_0.1.pt')

In [6]:
import h5py

# Path to the .h5 file
h5_file_path = "/home/tarun/rlogist/ACMIL/CAMELYON16_features/VIT_medical_ssl_level_1_3/lr/h5_files/patch_feats_pretrain_medical_ssl.h5"

# Specify the WSI filename whose coordinates you want
target_wsi = "test_038"  # Change this to your desired filename

# Open the .h5 file
with h5py.File(h5_file_path, "r") as h5_file:
    # Check available keys (optional, to understand the structure)
    print("Available keys:", list(h5_file.keys()))
    
    # Assuming the file structure has keys corresponding to WSI filenames
    if target_wsi in h5_file:
        # Extract coordinates
        coordinates = h5_file[target_wsi]["coords"][:]  # Assuming "coordinates" is a dataset inside each WSI group
        print(f"Coordinates for {target_wsi}:")
        print(coordinates)
    else:
        print(f"WSI {target_wsi} not found in the dataset.")

Available keys: ['normal_001', 'normal_002', 'normal_003', 'normal_004', 'normal_005', 'normal_006', 'normal_007', 'normal_008', 'normal_009', 'normal_010', 'normal_011', 'normal_012', 'normal_013', 'normal_014', 'normal_015', 'normal_016', 'normal_017', 'normal_018', 'normal_019', 'normal_020', 'normal_021', 'normal_022', 'normal_023', 'normal_024', 'normal_025', 'normal_026', 'normal_028', 'normal_029', 'normal_030', 'normal_031', 'normal_032', 'normal_033', 'normal_034', 'normal_035', 'normal_036', 'normal_037', 'normal_038', 'normal_039', 'normal_040', 'normal_041', 'normal_042', 'normal_043', 'normal_044', 'normal_045', 'normal_046', 'normal_047', 'normal_048', 'normal_049', 'normal_050', 'normal_051', 'normal_052', 'normal_053', 'normal_054', 'normal_055', 'normal_056', 'normal_057', 'normal_058', 'normal_059', 'normal_060', 'normal_061', 'normal_062', 'normal_063', 'normal_064', 'normal_065', 'normal_066', 'normal_067', 'normal_068', 'normal_069', 'normal_070', 'normal_071', 'no

In [7]:
len(coordinates), len(coordinates[0]), len(action_sequences['test_038'])

(414, 2, 83)

In [8]:
import openslide
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from shapely.geometry import Polygon
from matplotlib.patches import Polygon as MplPolygon, Rectangle
import imageio

# Paths
wsi_path = "/media/internal_8T/karm/karm_8T_backup/camelyon16/images/all/test_038.tif"
xml_path = "/media/internal_8T/karm/karm_8T_backup/camelyon16/annotations/test_038.xml"
output_gif = "highlighted_patches.gif"

# Load WSI
slide = openslide.OpenSlide(wsi_path)

# Get Level 3 (5X) properties
level = 3
downscale_factor = slide.level_downsamples[level]  # Scaling factor from level 0 to level 3
wsi_size = slide.level_dimensions[level]  # (width, height) at level 3

# Read WSI at level 3
wsi_img = slide.read_region((0, 0), level, wsi_size).convert("RGB")

# Parse XML for annotation coordinates
tree = ET.parse(xml_path)
root = tree.getroot()

annotations = []
for annotation in root.findall(".//Annotation"):
    coords = []
    for coord in annotation.findall(".//Coordinate"):
        x = float(coord.get("X")) / downscale_factor  # Scale to level 3
        y = float(coord.get("Y")) / downscale_factor  # Scale to level 3
        coords.append((x, y))
    annotations.append(Polygon(coords))  # Store as Shapely Polygon

# Example sequence of patches (list of coordinates in (x, y, width, height) format at level 0)
action = action_sequences['test_038']
patch_sequence = []
for id in action:
    patch_sequence.append(coordinates[id])


# Convert patch coordinates to Level 3
patch_sequence = [
    (x / downscale_factor, y / downscale_factor)
    for x, y in patch_sequence
]

# Create frames for the GIF
frames = []

for i, (x, y) in (tqdm(enumerate(patch_sequence), desc="Processing Patches", total=len(patch_sequence))):
    fig, ax = plt.subplots(figsize=(15, 15))
    ax.imshow(wsi_img)

    # Draw existing annotations
    for polygon in annotations:
        patch = MplPolygon(np.array(polygon.exterior.coords), edgecolor='r', fill=False, linewidth=2)
        ax.add_patch(patch)

    # Highlight the current patch with a green rectangle
    rect = Rectangle((x, y), 512, 512, edgecolor='lime', facecolor='none', linewidth=3)
    ax.add_patch(rect)

    ax.set_title(f"Highlighting Patch {i+1} at Level {level} (5X)")
    ax.axis("off")

    # Save frame to array
    fig.canvas.draw()
    frame = np.array(fig.canvas.renderer.buffer_rgba())
    frames.append(frame)
    plt.close(fig)

# Save as GIF
imageio.mimsave(output_gif, frames, duration=0.5)
print(f"GIF saved as {output_gif}")

Processing Patches:  86%|████████▌ | 71/83 [13:27<02:49, 14.08s/it]

: 

In [ ]:
import openslide
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from shapely.geometry import Polygon
from matplotlib.patches import Polygon as MplPolygon, Rectangle
import imageio
from tqdm import tqdm

# Paths
wsi_path = "/media/internal_8T/karm/karm_8T_backup/camelyon16/images/all/test_038.tif"
xml_path = "/media/internal_8T/karm/karm_8T_backup/camelyon16/annotations/test_038.xml"
output_gif_1 = "highlighted_patches_part1.gif"
output_gif_2 = "highlighted_patches_part2.gif"

# Load WSI
slide = openslide.OpenSlide(wsi_path)

# Get Level 3 (5X) properties
level = 3
downscale_factor = slide.level_downsamples[level]  # Scaling factor from level 0 to level 3
wsi_size = slide.level_dimensions[level]  # (width, height) at level 3

# Read WSI at level 3
wsi_img = slide.read_region((0, 0), level, wsi_size).convert("RGB")

# Parse XML for annotation coordinates
tree = ET.parse(xml_path)
root = tree.getroot()

annotations = []
for annotation in root.findall(".//Annotation"):
    coords = []
    for coord in annotation.findall(".//Coordinate"):
        x = float(coord.get("X")) / downscale_factor  # Scale to level 3
        y = float(coord.get("Y")) / downscale_factor  # Scale to level 3
        coords.append((x, y))
    annotations.append(Polygon(coords))  # Store as Shapely Polygon

# Extract patch sequence
action = action_sequences['test_038']
patch_sequence = [coordinates[id] for id in action]

# Convert patch coordinates to Level 3
patch_sequence = [(x / downscale_factor, y / downscale_factor) for x, y in patch_sequence]

# Split sequence into two parts
patch_sequence_1 = patch_sequence[:50]
patch_sequence_2 = patch_sequence[50:]

def create_gif(patch_sequence, output_gif):
    frames = []
    for i, (x, y) in tqdm(enumerate(patch_sequence), desc=f"Processing {output_gif}", total=len(patch_sequence)):
        fig, ax = plt.subplots(figsize=(15, 15))
        ax.imshow(wsi_img)

        # Draw existing annotations
        for polygon in annotations:
            patch = MplPolygon(np.array(polygon.exterior.coords), edgecolor='r', fill=False, linewidth=2)
            ax.add_patch(patch)

        # Highlight the current patch with a green rectangle
        rect = Rectangle((x, y), 512, 512, edgecolor='lime', facecolor='none', linewidth=3)
        ax.add_patch(rect)

        ax.set_title(f"Highlighting Patch {i+1} at Level {level} (5X)")
        ax.axis("off")

        # Save frame to array
        fig.canvas.draw()
        frame = np.array(fig.canvas.renderer.buffer_rgba())
        frames.append(frame)
        plt.close(fig)

    # Save as GIF
    imageio.mimsave(output_gif, frames, duration=0.5)
    print(f"GIF saved as {output_gif}")

# Create both videos
create_gif(patch_sequence_1, output_gif_1)
# create_gif(patch_sequence_2, output_gif_2)

NameError: name 'action_sequences' is not defined